In [0]:
!git clone "https://github.com/convman/Multimodal-MOSEI.git"

In [2]:
cd Multimodal-MOSEI/

/content/Multimodal-MOSEI


In [0]:
!wget "http://immortal.multicomp.cs.cmu.edu/raw_datasets/old_processed_data/cmu-mosei/labels/mosi2uni_Test_labels_surprise.csv"
!wget "http://immortal.multicomp.cs.cmu.edu/raw_datasets/old_processed_data/cmu-mosei/labels/mosi2uni_Train_labels_surprise.csv"


In [0]:
import pandas as pd
import numpy as np
import h5py
Test_labels_surprise = pd.read_csv("mosi2uni_Test_labels_surprise.csv",header=None)
Train_labels_surprise = pd.read_csv("mosi2uni_Train_labels_surprise.csv",header=None)

In [0]:
!wget "http://immortal.multicomp.cs.cmu.edu/raw_datasets/old_processed_data/cmu-mosei/data/audio_test.h5"
!wget "http://immortal.multicomp.cs.cmu.edu/raw_datasets/old_processed_data/cmu-mosei/data/audio_train.h5"
!wget "http://immortal.multicomp.cs.cmu.edu/raw_datasets/old_processed_data/cmu-mosei/data/text_test_emb.h5"
!wget "http://immortal.multicomp.cs.cmu.edu/raw_datasets/old_processed_data/cmu-mosei/data/text_train_emb.h5"
!wget "http://immortal.multicomp.cs.cmu.edu/raw_datasets/old_processed_data/cmu-mosei/data/video_test.h5"
!wget "http://immortal.multicomp.cs.cmu.edu/raw_datasets/old_processed_data/cmu-mosei/data/video_train.h5"

In [6]:
video_train = h5py.File("video_train.h5","r")
print(list(video_train.keys()))
video_train = np.array(video_train.get('d1'))
video_train.shape

video_test = h5py.File("video_test.h5","r")
print(list(video_test.keys()))
video_test = np.array(video_test.get('d1'))
video_test.shape

audio_train = h5py.File("audio_train.h5","r")
print(list(audio_train.keys()))
audio_train = np.array(audio_train.get('d1'))
audio_train.shape

audio_test = h5py.File("audio_test.h5","r")
print(list(audio_test.keys()))
audio_test = np.array(audio_test.get('d1'))
audio_test.shape

text_train_emb = h5py.File("text_train_emb.h5","r")
print(list(text_train_emb.keys()))
text_train_emb = np.array(text_train_emb.get('d1'))
text_train_emb.shape

text_test_emb = h5py.File("text_test_emb.h5","r")
print(list(text_test_emb.keys()))
text_test_emb = np.array(text_test_emb.get('d1'))
text_test_emb.shape


['d1']
['d1']
['d1']
['d1']
['d1']
['d1']


(4832, 20, 300)

In [0]:
from keras.models import load_model
import keras
from google.colab import files	
from keras.models import load_model
from keras.models import Model,Sequential,Model
from keras.layers import LSTM,Dense,Input,concatenate
from keras import callbacks
from keras.layers import *
from keras.callbacks import EarlyStopping, ModelCheckpoint

model_audio_surprise = load_model("unimodal baselines/weights/audio/weights_rnn_surprise.h5")
model_video_surprise = load_model("unimodal baselines/weights/video/weights_rnn_surprise.h5")
model_text_surprise = load_model("unimodal baselines/weights/text/weights_rnn_surprise.h5")


In [0]:
i1 = Input(shape=(20,35),name='i1')
i2 = Input(shape=(20,74),name='i2')
i3 = Input(shape=(20,300),name='i3')

o1 = model_video_surprise(i1)
o2 = model_audio_surprise(i2)
o3 = model_text_surprise(i3)

In [0]:
def deep_fuse(out):
  o1 = out[0]     # o1.shape = (None,32)
  o2 = out[1]     # o2.shape = (None,32)
  o3 = out[2]     # o3.shape = (None,32)

  x = 1.-o1
  y = 1.-o2
  z = 1.-o3

  return concatenate([o1,o2,o3,x,y,z])


In [0]:
merged = Lambda(deep_fuse)([o1,o2,o3])
y = Dense(32,activation='relu')(merged)
y = Dropout(rate=0.15)(y)
y = Dense(32,activation='relu')(y)
y = Dense(1,activation='sigmoid')(y)

In [16]:
model = Model(inputs=[i1,i2,i3],outputs=y)
model.compile('adam','binary_crossentropy',metrics=['accuracy'])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
i1 (InputLayer)                 (None, 20, 35)       0                                            
__________________________________________________________________________________________________
i2 (InputLayer)                 (None, 20, 74)       0                                            
__________________________________________________________________________________________________
i3 (InputLayer)                 (None, 20, 300)      0                                            
__________________________________________________________________________________________________
model_10 (Model)                (None, 1)            1258337     i1[0][0]                         
__________________________________________________________________________________________________
model_4 (M

In [17]:
es = EarlyStopping(monitor='val_loss',mode='min' ,patience=5, min_delta=0.0001,verbose=1)
mcp = ModelCheckpoint("deepfuison_surprise.h5",monitor='val_loss',verbose=1)
model.fit({'i1':video_train,'i2':audio_train,'i3':text_train_emb},Train_labels_surprise,128,20,validation_split=0.1,callbacks=[es, mcp])
model.evaluate({'i1':video_test,'i2':audio_test,'i3':text_test_emb},Test_labels_surprise)

Train on 13761 samples, validate on 1529 samples
Epoch 1/20
13761/13761 [==============================] - 116s 8ms/step - loss: 0.3377 - acc: 0.8825 - val_loss: 0.3399 - val_acc: 0.8836

Epoch 00001: saving model to deepfuison_surprise.h5
Epoch 2/20
13761/13761 [==============================] - 87s 6ms/step - loss: 0.1056 - acc: 0.9681 - val_loss: 0.4524 - val_acc: 0.8620

Epoch 00002: saving model to deepfuison_surprise.h5
Epoch 3/20
13761/13761 [==============================] - 118s 9ms/step - loss: 0.0607 - acc: 0.9829 - val_loss: 0.4889 - val_acc: 0.8869

Epoch 00003: saving model to deepfuison_surprise.h5
Epoch 4/20
13761/13761 [==============================] - 126s 9ms/step - loss: 0.0414 - acc: 0.9882 - val_loss: 0.5716 - val_acc: 0.8770

Epoch 00004: saving model to deepfuison_surprise.h5
Epoch 5/20
13761/13761 [==============================] - 121s 9ms/step - loss: 0.0380 - acc: 0.9885 - val_loss: 0.5618 - val_acc: 0.8679

Epoch 00005: saving model to deepfuison_surprise.

[0.7027797106574813, 0.8797599337748344]

In [0]:
files.download("deepfuison_surprise.h5")